In [17]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv3D, Conv1D, MaxPooling2D,Dropout,UpSampling2D,UpSampling3D,concatenate,Conv2DTranspose,Add
from tensorflow.keras import Sequential
from keras.layers import Dense, Flatten

In [24]:
inputs = tf.keras.Input(shape=( None, 256,256, 1))
conv_layer = Conv3D(16, (3,3,3), activation = 'relu', padding = 'same')(inputs)
conv_layer1 = Conv3D(16, (3,3,3), activation = 'relu', padding = 'same')(conv_layer)
pool1 =  Dropout(0.3)(conv_layer1)
conv_layer1 = Conv3D(16, (3,3, 3), activation = 'relu', padding = 'same')(pool1)
conv_layer1 = Add()([conv_layer, conv_layer1])

conv_layer1 = Conv3D(32, (3,3, 3), activation = 'relu', padding = 'same', strides=(2,2, 2))(conv_layer1)

conv_layer2 = Conv3D(32,(3,3,3) , activation = 'relu', padding = 'same')(conv_layer1)
pool2 =  Dropout(0.3)(conv_layer2)
conv_layer2 = Conv3D(32, (3,3,3), activation = 'relu', padding = 'same')(pool2)
conv_layer2 = Add()([conv_layer1, conv_layer2])

conv_layer2 = Conv3D(64, (3,3, 3), activation = 'relu', padding = 'same', strides=(2,2, 2))(conv_layer2)

conv_layer3 = Conv3D(64,(3,3,3) , activation = 'relu', padding = 'same')(conv_layer2)
pool3 =  Dropout(0.3)(conv_layer3)
conv_layer3 = Conv3D(64, (3,3,3), activation = 'relu', padding = 'same')(pool3)
conv_layer3 = Add()([conv_layer2, conv_layer3])

conv_layer3 = Conv3D(128, (3,3, 3), activation = 'relu', padding = 'same', strides=(2,2, 2))(conv_layer3)

conv_layer4 = Conv3D(128,(3,3,3) , activation = 'relu', padding = 'same')(conv_layer3)
pool4 =  Dropout(0.3)(conv_layer4)
conv_layer4 = Conv3D(128, (3,3,3), activation = 'relu', padding = 'same')(pool4)
conv_layer4 = Add()([conv_layer3, conv_layer4])


conv_layer4 = Conv3D(256, (3,3, 3), activation = 'relu', padding = 'same', strides=(2,2, 2))(conv_layer4)

conv_layer5 = Conv3D(256,(3,3,3) , activation = 'relu', padding = 'same')(conv_layer4)
pool5 =  Dropout(0.3)(conv_layer5)
conv_layer5 = Conv3D(256, (3,3,3), activation = 'relu', padding = 'same')(pool5)
conv_layer5 = Add()([conv_layer4, conv_layer5])

layer_6 = UpSampling3D(128,(3,3,3) , activation = 'relu',data_format= , padding = 'same')(conv_layer5)

TypeError: __init__() got multiple values for argument 'data_format'